In [81]:
%load_ext autoreload
%autoreload 2
from data_translator import CMAToScheduleTranslator, TestTranslator, EncodeForCMA
from hybrid_solution_data_loader import get_data
from models import Order, SimulationEnvironment, Schedule
from solver import CMAWeightSolver
from visualize import visualize_schedule_demo
import objective_function

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [82]:
n_workstations, recipes, operation_times = get_data(3)
recipies, workstations, resources, tasks, _ = TestTranslator().translate(n_workstations, recipes, operation_times)

env = SimulationEnvironment(workstations, tasks, resources, recipies)

In [83]:
earliest_slot = 0
last_slot = 100
recipe_orders = [0, 1, 2, 3, 0, 3, 2, 1, 0, 3] # for dataset 0
orders = []
o_id = 0
for order in recipe_orders:
    orders.append(Order(o_id, 0, last_slot, last_slot, [order], 100, 50, False, 0, False, 500)) # for now: use resources to select recipe
    o_id = o_id + 1

In [84]:
encoder : EncodeForCMA = EncodeForCMA()
values, durations, jobs, due_dates = encoder.translate(env, orders)

In [85]:
solver : CMAWeightSolver = CMAWeightSolver(values, durations, jobs, env, orders)
solver.initialize() # doesn't do anything right now
solver.run()

(7_w,15)-aCMA-ES (mu_w=4.5,w_1=34%) in dimension 40 (seed=946713, Mon Jan 16 15:36:46 2023)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     15 1.010000000000000e+02 1.0e+00 4.67e-01  5e-01  5e-01 0:13.5
    2     30 1.010000000000000e+02 1.0e+00 4.46e-01  4e-01  4e-01 0:27.0
    3     45 1.010000000000000e+02 1.1e+00 4.39e-01  4e-01  4e-01 0:41.5
    4     60 1.010000000000000e+02 1.1e+00 4.32e-01  4e-01  4e-01 0:55.2
    5     75 1.010000000000000e+02 1.1e+00 4.24e-01  4e-01  4e-01 1:07.7
    6     90 1.010000000000000e+02 1.1e+00 4.22e-01  4e-01  4e-01 1:20.6
    7    105 1.010000000000000e+02 1.1e+00 4.22e-01  4e-01  4e-01 1:33.1
    8    120 1.010000000000000e+02 1.1e+00 4.13e-01  4e-01  4e-01 1:45.5
    9    135 1.010000000000000e+02 1.2e+00 4.08e-01  4e-01  4e-01 1:57.9
   10    150 1.010000000000000e+02 1.2e+00 4.05e-01  4e-01  4e-01 2:09.9
termination on tolfunhist=1e-12 (Mon Jan 16 15:38:56 2023)
final/bestever f-value = 1.010000e+02 1.010000e

In [108]:
result = solver.best_solution[0]
print("Parameters of the best solution :\n {solution}".format(solution=result))
print(f'Fitness value of the solution: {solver.best_solution[1].result.fbest}')

Parameters of the best solution :
 [0.84234805 0.31139179 0.39138281 0.08508902 0.03635738 0.39237878
 0.43353286 0.04332257 0.16807396 0.14474499 0.04267942 0.70965012
 0.22036514 0.2553376  0.02004393 0.44771925 0.56632169 0.17077751
 0.0151532  0.00121217 0.04150775 0.34435233 0.53274464 0.33964387
 0.00693424 0.49525923 0.14747666 0.39032867 0.57129414 0.65962834
 0.01332299 0.17107172 0.46047647 0.12045988 0.76805993 1.09115626
 0.44702113 0.3110043  0.11283845 0.0014272 ]
Fitness value of the solution: 101


In [105]:
schedule = CMAToScheduleTranslator().translate(result, jobs, due_dates, durations, env, orders)
schedule.created_in = env
schedule.created_for = orders
schedule.created_by = solver

In [99]:
#TODO: current solution could be a result of either a) bug in schedule building (more likely) or b) lack of feasibility checks, since they shouldn't be necessary
visualize_schedule_demo(schedule, env, orders)

In [100]:
fitness_values = objective_function.calculate_comparison_values(schedule, orders, env)
print(f'Fitness Values:\nMakespan: {fitness_values[0]}\nTardiness: {fitness_values[1]}\nDeviation: {fitness_values[2]}\nIdle Time: {fitness_values[3]}\nProfit: {fitness_values[4]}')

AttributeError: 'NoneType' object has no attribute 'tasks'